In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
from econml.utilities import _safe_norm_ppf


def dml_sim(model_t, model_y, n_folds, Y, T, X):
    """
    DML for a single treatment and outcome.
 
    Parameters
    ----------
    model_t : object
        A fitted model object for the treatment.
    model_y : object
        A fitted model object for the outcome.
    n_folds : int
        The number of folds to use in cross-validation.
    Y : array-like
        The outcome variable.
    T : array-like
        The treatment variable.
    X : array-like
        The covariates.
 
    Returns
    -------
    float
        The estimated treatment effect.
    """
 
    # Initialize KFold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True)
 
    # Initialize arrays to hold predictions
    y_res = np.zeros_like(Y)
    t_res = np.zeros_like(T)
 
    # Cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        T_train, T_test = T[train_index], T[test_index]
 
        # Fit the treatment model
        t_res[test_index] = T_test-model_t.fit(X_train, T_train).predict(X_test).reshape(T_test.shape)
 
        # Fit the outcome model
        y_res[test_index] = Y_test-model_y.fit(X_train, Y_train).predict(X_test).reshape(Y_test.shape)

    # for testing.. use econml nuisances
    # y_res = econ_y_res.reshape(-1, 1)  # Reshape y_res to be a column vector
    # t_res = econ_t_res.reshape(-1, 1)  # Reshape t_res to be a column vector
 
    # ATE, sigma^2, and nu^2
    

    # print('rmse t', np.mean(t_res**2)**0.5)
    # print('rmse y:', np.mean(y_res**2)**0.5)

    # smlr = StatsModelsLinearRegression(fit_intercept=False).fit(t_res, y_res)
    # theta_smlr = smlr.coef_[0]
    # var_smlr = smlr._var[0][0]
    # print('theta_smlr:', theta_smlr)
    # print('var_smlr:', var_smlr)
    # print('se_smlr:', np.sqrt(var_smlr))
 
    theta = np.mean(y_res*t_res) / np.mean(t_res**2)  # Estimate the treatment effect
    # print('theta:', theta)
    sigma2 = np.mean((y_res - theta*t_res)**2)  # Estimate the variance of the outcome residuals (after subtracting the treatment effect)
    nu2 = 1/np.mean(t_res**2)  # Estimate the variance of the treatment
    ests = np.array([theta, sigma2, nu2])  # Estimated parameters
 
    ls = np.concatenate([t_res**2, np.ones_like(t_res), t_res**2], axis=1)
 
    G = np.diag(np.mean(ls, axis=0))  # G matrix, diagonal with means of ls
    G_inv = np.linalg.inv(G)  # Inverse of G matrix, could just take reciprocals since it's diagonal
 
 
    residuals = np.concatenate([y_res*t_res-theta*t_res*t_res, (y_res-theta*t_res)**2-sigma2, t_res**2*nu2-1], axis=1)  # Combine residuals
    Ω = residuals.T @ residuals / len(residuals)  # Estimate the covariance matrix of the residuals
    cov = G_inv @ Ω @ G_inv / len(residuals)  # Estimate the variance of the parameters
 
    return theta, sigma2, nu2, cov

def sensitivity_interval(theta, sigma, nu, cov, alpha, c_y, c_t, rho):
    # [theta, sigma, nu] = ests
    C = np.abs(rho) * np.sqrt(c_y) * np.sqrt(c_t/(1-c_t)) / 2
    ests = np.array([theta, sigma, nu])
    coefs_p = np.array([1, C*np.sqrt(nu/sigma), C*np.sqrt(sigma/nu)])
    coefs_n = np.array([1, -C*np.sqrt(nu/sigma), -C*np.sqrt(sigma/nu)])
    # One dimensional normal distribution:
    sigma_p = coefs_p @ cov @ coefs_p
    sigma_n = coefs_n @ cov @ coefs_n

    lb = _safe_norm_ppf(alpha / 2, loc=ests @ coefs_n, scale=np.sqrt(sigma_n))
    ub = _safe_norm_ppf(1 - alpha / 2, loc=ests @ coefs_p, scale=np.sqrt(sigma_p))

    return (lb, ub)

def RV(theta, sigma, nu, cov, alpha):
    # The robustness value is the degree of confounding of *both* the treatment and the outcome that still produces an interval
    # that excludes zero.

    # We're looking for a value of r such that the sensitivity bounds just touch zero

    r = 0
    r_up = 1
    r_down = 0
    lb, ub = sensitivity_interval(theta, sigma, nu, cov, alpha, 0, 0, 1)
    if lb < 0 and ub > 0:
        return 0
    
    else:
        if lb > 0:
            target = 0
            mult = 1
            d = lb
        else:
            target = 1
            mult = -1
            d = ub

    while abs(d) > 1e-6:
        d = mult * sensitivity_interval(theta, sigma, nu, cov, alpha, r, r, 1)[target]
        if d > 0:
            r_down = r
        else:
            r_up = r

        r = (r_down + r_up) / 2
        
    return r

    


# Simulate data

In [2]:
from sklearn.linear_model import LinearRegression

n = 1000

alpha = np.random.normal(size=30)
beta = np.random.normal(size=30)

X = np.random.normal(size=(n,30))

t = np.random.normal(size=n) + 2 * X @ alpha
y = np.random.normal(size=n) + 3*t + 50* X @ beta

T = t.reshape(-1, 1)
Y = y.reshape(-1, 1)



# Simple example

Run simple dml and calculate intermediate values

In [3]:
sig_level = 0.1

theta, sigma, nu, sig = dml_sim(LinearRegression(), LinearRegression(), 2, Y, T, X)


#### Calculate a "sensitivity interval". 

Need to supply "strength of latent confounder" as argument.

In [4]:
sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.6, 0.6, 1)

(1.9713167396783304, 3.992013510060846)

#### Calculate Robustness Value. 

The required strength of a latent confounder in order for the confidence interval to include 0.

In [5]:
RV(theta, sigma, nu, sig, sig_level)

0.9007053971290588

# Ablations

In [6]:
sig_level = 0.1

results = []
for i in [1, 5, 15, 30]:
    theta, sigma, nu, sig = dml_sim(LinearRegression(), LinearRegression(), 2, Y, T, X[:,:i])
    result_dict = {
        'i': i,
        'alpha': sig_level,
        'sensitivity_interval': sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.6, 0.6, 1),
        'RV': RV(theta, sigma, nu, sig, sig_level)
    }
    results.append(result_dict)
                   
results[-1]

{'i': 30,
 'alpha': 0.1,
 'sensitivity_interval': (2.006244806224368, 4.0212629919887455),
 'RV': 0.9028782248497009}

In [8]:
(
    pd.DataFrame(results)
)

,i,alpha,sensitivity_interval,RV
0,1,0.1,"(-5.76103393577295, 30.16877869667943)",0.463505
1,5,0.1,"(-4.254207344618898, 31.474674283379382)",0.503265
2,15,0.1,"(2.926606519518738, 25.91115187623303)",0.679077
3,30,0.1,"(2.006244806224368, 4.0212629919887455)",0.902878


# DoubleML

In [9]:
import doubleml as dml
import pandas as pd
import numpy as np

In [10]:
df = pd.concat([
    pd.DataFrame(Y).squeeze().to_frame('Y'),
    pd.DataFrame(T).squeeze().to_frame('T'),
    pd.DataFrame(X).add_prefix('X'),
], axis=1)

df.head()

,Y,T,X0,X1,X2,X3,X4,X5,X6,X7,...,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29
0,169.161860,7.570734,-1.720637,-0.480190,1.793066,-0.909049,-0.491242,0.407515,-2.719292,0.214566,...,-1.236440,0.460231,-0.266955,0.185816,0.729663,1.324584,-0.091459,1.387470,-0.982113,-2.001484
1,-368.478684,-10.259294,1.502222,-0.351112,0.326909,0.383989,1.019918,0.102063,-1.172437,2.210359,...,-1.044021,-0.816131,0.288329,-0.428007,1.047723,-0.058769,-0.664951,-0.111616,1.111973,0.214340
2,595.208775,3.414269,-0.584506,-0.410095,-0.085977,0.515477,1.374335,1.912329,-0.645654,-0.413862,...,-0.029845,-1.035931,-0.146995,-1.224990,-2.704679,1.110162,-0.191411,0.852031,-0.457792,-2.563874
3,265.170499,21.740763,0.285862,-0.802048,0.211192,-0.799453,0.643080,-0.379773,0.055227,1.142309,...,-0.290620,-1.175747,0.404689,-1.427746,0.069783,0.237225,-0.983619,0.459482,-0.306016,1.853582
4,66.916752,12.252024,-1.129366,-1.768697,-1.095474,0.536305,1.300771,0.316005,0.877389,-0.117935,...,-0.062981,0.789482,1.940387,1.153639,0.741752,-0.210623,0.046603,1.389971,-0.172416,0.265279


In [11]:
dml_data = dml.DoubleMLData(df, 'Y', 'T')
dml_data

In [12]:
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=LinearRegression(),
                          ml_m=LinearRegression(),
                          n_folds=2,
                          score='partialling out',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: Y
Treatment variable(s): ['T']
Covariates: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29']
Instrument variable(s): None
No. Observations: 1000

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: LinearRegression()
Learner ml_m: LinearRegression()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[3.19708721]]
Learner ml_m RMSE: [[1.01678607]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err          t  P>|t|     2.5 %    97.5 %
T  2.992753  0.032487  92.121737    

In [13]:
dml_obj.sensitivity_analysis(cf_y=0.03, cf_d=0.03, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0  2.909926     2.963376  2.992753     3.022129  3.075592

------------------ Robustness Values ------------------
   H_0     RV (%)    RVa (%)
0  0.0  91.336809  90.546872


In [14]:
sens_benchmark = dml_obj.sensitivity_benchmark(benchmarking_set=["X6"])
print(sens_benchmark)


   cf_y     cf_d  rho  delta_theta
T   1.0  0.13301 -1.0    -7.112185


# New datasets

### DoubleML confounded synthetic data

In [15]:
from doubleml.datasets import make_confounded_plr_data

In [16]:
cf_y = 0.1
cf_d = 0.1
theta = 5.0
dpg_dict = make_confounded_plr_data(n_obs=10000, cf_y=cf_y, cf_d=cf_d, theta=theta)

In [17]:
x_cols = [f'X{i + 1}' for i in np.arange(dpg_dict['x'].shape[1])]
df = pd.DataFrame(np.column_stack((dpg_dict['x'], dpg_dict['y'], dpg_dict['d'])), columns=x_cols + ['y', 'd'])
dml_data = dml.DoubleMLData(df, 'y', 'd')

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=RandomForestRegressor(),
                          ml_m=RandomForestRegressor(),
                          n_folds=2,
                          score='partialling out',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4']
Instrument variable(s): None
No. Observations: 10000

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor()
Learner ml_m: RandomForestRegressor()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[8.43266256]]
Learner ml_m RMSE: [[1.13959992]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err          t  P>|t|     2.5 %    97.5 %
d  4.386734  0.076845  57.085535    0.0  4.236121  4.537348


In [20]:
dml_obj.sensitivity_analysis(cf_y=0.1, cf_d=0.1, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.1; cf_d=0.1, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0  3.652547     3.758583  4.386734     5.014886  5.170425

------------------ Robustness Values ------------------
   H_0     RV (%)    RVa (%)
0  0.0  51.346635  49.604976


In [21]:
dpg_dict['y'].shape

(10000,)

In [22]:
sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.1, 0.1, 1)

(4.84204509497674, 5.158534499404629)

In [23]:
sig_level=0.05

theta, sigma, nu, sig = dml_sim(
    RandomForestRegressor(), 
    RandomForestRegressor(), 
    2, 
    dpg_dict['y'].reshape(-1, 1),
    dpg_dict['d'].reshape(-1, 1),
    dpg_dict['x']
)

result_dict = {
    'alpha': sig_level,
    'sig': sig,
    'sensitivity_interval': sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.6, 0.6, 1),
    'RV': RV(theta, sigma, nu, sig, sig_level)
}
result_dict

c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: Dat

{'alpha': 0.05,
 'sig': array([[ 5.97407504e-03, -1.67034912e-01, -5.43651824e-05],
        [-1.67034912e-01,  1.20203175e+01,  2.52109790e-03],
        [-5.43651824e-05,  2.52109790e-03,  1.18426109e-04]]),
 'sensitivity_interval': (-1.6122852078135446, 10.442290925119165),
 'RV': 0.4936022162437439}

In [24]:
result_dict['RV']

0.4936022162437439

### 401k data

In [25]:
dml_data = dml.datasets.fetch_401K()

In [26]:
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=RandomForestRegressor(),
                          ml_m=RandomForestRegressor(),
                          n_folds=5,
                          score='partialling out',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: net_tfa
Treatment variable(s): ['e401']
Covariates: ['age', 'inc', 'educ', 'fsize', 'marr', 'twoearn', 'db', 'pira', 'hown']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor()
Learner ml_m: RandomForestRegressor()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[56554.88810261]]
Learner ml_m RMSE: [[0.46880446]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
             coef      std err         t         P>|t|        2.5 %  \
e401  9545.723974  1260.808673  7.571112  3.700422e-14  7074.584382   

            97.5 %  
e401  12016.863565  


In [27]:
dml_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.04, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.04; cf_d=0.04, rho=1.0

------------------ Bounds with CI    ------------------
      CI lower  theta lower        theta   theta upper      CI upper
0  2409.550717  4636.205456  9545.723974  14455.242491  16511.651393

------------------ Robustness Values ------------------
   H_0    RV (%)   RVa (%)
0  0.0  7.628916  5.816637


In [30]:
from sklearn.linear_model import LinearRegression, LassoCV

In [ ]:
sig_level=0.05

theta, sigma, nu, sig = dml_sim(
    RandomForestRegressor(), 
    RandomForestRegressor(), 
    5, 
    dml_data.y.reshape(-1, 1),
    dml_data.d.reshape(-1, 1),
    dml_data.x
)

# seem to get weird results here but not when I pass econml nuisances directly inside dml_sim func.
result_dict = {
    'alpha': sig_level,
    'sensitivity_interval': sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.00001, 0.0001, 1),
    'RV': RV(theta, sigma, nu, sig, sig_level)
}
result_dict

c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: Dat

{'alpha': 0.05,
 'sensitivity_interval': (-3259.2844095062455, 7370.312469068845),
 'RV': 0}

In [ ]:
from econml.dml import LinearDML

est = LinearDML(model_y=RandomForestRegressor(), model_t=RandomForestRegressor()).fit(
    Y=dml_data.y, T=dml_data.d, W=dml_data.x, cache_values=True
)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,8923.022,1328.008,6.719,0.0,6320.175,11525.87


store econml nuisances to use inside dml_sim func for debugging

In [40]:
econ_y_res, econ_t_res = est._cached_values.nuisances

### bonus data

In [33]:
dml_data = dml.datasets.fetch_bonus()

In [34]:
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=RandomForestRegressor(),
                          ml_m=RandomForestRegressor(),
                          n_folds=2,
                          score='partialling out',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: inuidur1
Treatment variable(s): ['tg']
Covariates: ['female', 'black', 'othrace', 'dep1', 'dep2', 'q2', 'q3', 'q4', 'q5', 'q6', 'agelt35', 'agegt54', 'durable', 'lusd', 'husd']
Instrument variable(s): None
No. Observations: 5099

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor()
Learner ml_m: RandomForestRegressor()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[1.28984655]]
Learner ml_m RMSE: [[0.5034491]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
        coef   std err         t     P>|t|     2.5 %    97.5 %
tg -0.079585  0.035971 -2.212475  0.026934 -0.150086 -0.009083


In [35]:
dml_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.04, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.04; cf_d=0.04, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0 -0.243371    -0.184128 -0.079585     0.024959  0.084101

------------------ Robustness Values ------------------
   H_0    RV (%)   RVa (%)
0  0.0  3.059967  0.794663


In [36]:
sig_level=0.05

theta, sigma, nu, sig = dml_sim(
    RandomForestRegressor(), 
    RandomForestRegressor(), 
    5, 
    dml_data.y.reshape(-1, 1),
    dml_data.d.reshape(-1, 1),
    dml_data.x
)

result_dict = {
    'alpha': sig_level,
    'sensitivity_interval': sensitivity_interval(theta, sigma, nu, sig, sig_level, 0.05, 0.05, 1),
    'RV': RV(theta, sigma, nu, sig, sig_level)
}
result_dict

c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\sklearn\base.py:1473: Dat

{'alpha': 0.05,
 'sensitivity_interval': (-1.9404570165795167, 0.9777681976085163),
 'RV': 0}